In [ ]:
%%time
%%capture pip
!sudo apt update && sudo apt install python3-pip

In [ ]:
%%time
%%capture pip_hagrid
!pip install -U hagrid

In [ ]:
%%time
%%capture hagrid
!hagrid launch to docker:80 --tag=latest --tail=false

In [ ]:
%%time
!hagrid check --wait --silent

In [ ]:
%%time
%%capture syft
!pip install -q --pre syft